In [1]:
import os
import numpy as np
import pandas as pd

project_folder = r'D:\issp_data\feature_csv'
arr_X, arr_y = np.ones(10).reshape((-1, 10)), np.array([1])

for fl in os.listdir(project_folder):
    pa = os.path.join(project_folder, fl)
    _ = pd.read_csv(pa).values
    arr_X = np.vstack((arr_X, _))
    arr_y = np.hstack((arr_y, np.array([int(fl[31])] * _.shape[0])))
    
arr_X, arr_y = arr_X[1:, ], arr_y[1:]
print(arr_X)
print(arr_y)

[[1.48331343e+09 1.48323007e+09 8.33620000e+04 ... 5.64170404e+01
  1.60000000e+01 1.14833204e+02]
 [1.48455206e+09 1.48455100e+09 1.06000000e+03 ... 2.60000000e+01
  7.00000000e+00 3.50218979e+01]
 [1.48352021e+09 1.48351758e+09 2.63000000e+03 ... 2.36250000e+01
  1.65000000e+01 2.54833483e+01]
 ...
 [1.51388452e+09 1.51381656e+09 6.79560000e+04 ... 3.38450000e+02
  1.30500000e+02 4.96051960e+02]
 [1.51346061e+09 1.51343285e+09 2.77630000e+04 ... 1.40525000e+03
  7.52000000e+02 1.57189748e+03]
 [1.51249285e+09 1.51246326e+09 2.95860000e+04 ... 7.00000000e+01
  3.20000000e+01 6.64018072e+01]]
[0 0 0 ... 2 2 2]


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils import resample
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [3]:
X_train, X_test, y_train, y_test = train_test_split(
    arr_X, arr_y, test_size=0.2, stratify=arr_y, random_state=None)
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

X_train, y_train = resample(X_train, y_train, n_samples=X_train.shape[0]//300,
                            replace=False, stratify=y_train, random_state=None)
X_test, y_test = resample(X_test, y_test, n_samples=X_test.shape[0]//300,
                          replace=False, stratify=y_test, random_state=None)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(6643944, 10) (1660987, 10)
(6643944,) (1660987,)
(22146, 10) (22146,)
(5536, 10) (5536,)


In [ ]:
# SVC, too slow and poor accuracy, so ignore it!
# clf = SVC(gamma='auto', verbose=2)
# clf.fit(X_train, y_train)
# y_pred = clf.predict(X_test)
# print(clf.score(X_test, y_test))
# print(confusion_matrix(y_test, y_pred))

In [6]:
# Random forest
clf_2 = RandomForestClassifier(verbose=2)
clf_2.fit(X_train, y_train)
y_pred = clf_2.predict(X_test)
print(clf_2.score(X_test, y_test))
print(confusion_matrix(y_test, y_pred))

C:\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


building tree 1 of 10
building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    1.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0.6735910404624278
[[2327  363  115]
 [ 613  805  201]
 [ 230  285  597]]


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished


In [14]:
def get_config():
    config = {}
    ca_config = {}
    ca_config['random_state'] = 0
    ca_config['max_layers'] = 100
    ca_config['early_stopping_rounds'] = 3
#     ca_config["n_classes"] = 10
    ca_config['n_classes'] = 3
    ca_config['estimators'] = []
    ca_config['estimators'].append({
        'n_folds': 5, 
        'type': 'XGBClassifier', 
        'n_estimators': 10, 
        'max_depth': 5,
        'objective': 'multi:softprob', 
#         'silent': True, 
        'silent': False, 
        'nthread': -1, 
        'learning_rate': 0.1
    })
#     ca_config['estimators'].append({
#         'n_folds': 5, 
#         'type': 'RandomForestClassifier', 
#         'n_estimators': 10, 
#         'max_depth': None, 
#         'n_jobs': -1
#     })
    ca_config["estimators"].append({
        'n_folds': 5, 
        'type': 'ExtraTreesClassifier', 
        'n_estimators': 10, 
        'max_depth': None, 
        'n_jobs': -1
    })
#     ca_config["estimators"].append({
#         'n_folds': 5, 
#         'type': 'LogisticRegression'
#     })
    config["cascade"] = ca_config
    
    return config

In [15]:
# gcForest
import sys
sys.path.insert(0, r'gcForest\lib')
from gcforest.gcforest import GCForest

config = get_config()
gc = GCForest(config) # should be a dict
# gc = GCForest()
X_train_enc = gc.fit_transform(X_train, y_train)
y_pred = gc.predict(X_test)

[ 2019-10-30 13:41:21,913][cascade_classifier.fit_transform] X_groups_train.shape=[(22146, 10)],y_train.shape=(22146,),X_groups_test.shape=no_test,y_test.shape=no_test
[ 2019-10-30 13:41:21,920][cascade_classifier.fit_transform] group_dims=[10]
[ 2019-10-30 13:41:21,923][cascade_classifier.fit_transform] group_starts=[0]
[ 2019-10-30 13:41:21,925][cascade_classifier.fit_transform] group_ends=[10]
[ 2019-10-30 13:41:21,927][cascade_classifier.fit_transform] X_train.shape=(22146, 10),X_test.shape=(0, 10)
[ 2019-10-30 13:41:21,972][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(22146, 10), X_cur_test.shape=(0, 10)
[ 2019-10-30 13:41:22,967][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 5_folds.train_0.predict)=70.05%
[ 2019-10-30 13:41:23,803][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 5_folds.train_1.predict)=69.28%
[ 2019-10-30 13:41:24,723][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 5_folds.

[ 2019-10-30 13:42:00,921][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 5_folds.train_cv.predict)=69.36%
[ 2019-10-30 13:42:01,436][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_1 - 5_folds.train_0.predict)=66.52%
[ 2019-10-30 13:42:01,868][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_1 - 5_folds.train_1.predict)=66.46%
[ 2019-10-30 13:42:02,336][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_1 - 5_folds.train_2.predict)=66.29%
[ 2019-10-30 13:42:02,817][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_1 - 5_folds.train_3.predict)=66.38%
[ 2019-10-30 13:42:03,317][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_1 - 5_folds.train_4.predict)=67.10%
[ 2019-10-30 13:42:03,322][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_1 - 5_folds.train_cv.predict)=66.55%
[ 2019-10-30 13:42:03,327][cascade_classifier.calc_accuracy] Accuracy(layer_4 - train.classifier_average)=68.35%
[ 2019-10-30 13:4

In [16]:
print(X_train_enc)
print(y_pred)

[[0.52522033 0.29208654 0.18269314 0.5        0.5        0.        ]
 [0.61395633 0.21771622 0.16832742 0.9        0.1        0.        ]
 [0.68931395 0.15765026 0.1530358  1.         0.         0.        ]
 ...
 [0.66884786 0.17649694 0.1546552  0.9        0.1        0.        ]
 [0.1739515  0.21009067 0.6159578  0.         0.2        0.8       ]
 [0.34427795 0.45491296 0.20080912 0.4        0.5        0.1       ]]
[1 1 2 ... 0 0 1]
